# Improve C# Code Performance 

- [reference](https://medium.com/stackademic/stop-writing-slow-c-span-and-memory-are-the-upgrade-your-codebase-desperately-needs-d6d61240682b). 

In [ ]:
using System;

var input = "123,45.67";
var parts = input.Split(','); //Split allocates memory for a new array, Each part becomes a new string in memory
//modern dotnet use splice with span to avoid allocations
var id = int.Parse(parts[0]);
var score = double.Parse(parts[1]);

Console.WriteLine($"ID: {id}, Score: {score}");

ID: 123, Score: 45.67


- Span<T> is a stack-only type that represents a contiguous region of arbitrary memory.  
- It can be used to work with slices of arrays, strings, or unmanaged memory without additional allocations.
- Using Span<T> can help improve performance by reducing memory allocations and garbage collection overhead, especially in high-performance scenarios.


In [17]:
{

    ReadOnlySpan<char> span = input.AsSpan(); // var 키워드 사용으로 컴파일러가 타입 추론
    var prefix = span.Split(",");
    foreach(var item in prefix)
    {
        Console.WriteLine(span[item].ToString());
    }
}

123
45.67


In [ ]:
//Span Enumerator Example

using System;
using System.Threading.Tasks;

public class Program
{
    private static readonly byte[] _array = new byte[5];

    public static void Main()
    {
        new Random(42).NextBytes(_array);
        Span<byte> span = _array;

        Task.Run( () => ClearContents() );

       EnumerateSpan(span);
    }

    public static void ClearContents()
    {
        Task.Delay(20).Wait();
        lock (_array)
        {
           Array.Clear(_array, 0, _array.Length);
        }
    }

    public static void EnumerateSpan(Span<byte> span)
    {
        lock (_array)
        {
            foreach (byte element in span) //Span<T>.Enumerator is a ref struct.
            {
                Console.WriteLine(element);
                Task.Delay(10).Wait();
            }
        }

    }
}
// The example displays output like the following:
//     62
//     23
//     186
//     0
//     0
Program.Main();

62
23
186
150
174
